In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext tensorboard

In [3]:
import numpy as np
import tensorflow as tf

from alphazero import tools as gt

In [4]:
input_size=7

In [5]:
! env | grep LD_

In [6]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [7]:
import os
import tempfile
from pathlib import Path

tempdir = tempfile.mkdtemp()

data_dir = Path(os.environ.get('MODELS_DIR', tempdir)) / 'gomoku'
models_dir = data_dir / 'models'
logs_dir = data_dir / 'logs'
cp_dir = data_dir / 'checkpoints'
print(f"Checkpoints can be found in {cp_dir}")
print(f"Models can be found in {models_dir}")
print(f"Logs can be found in {logs_dir}")

Checkpoints can be found in /var/folders/0s/fny92xb54c50mb90g3l_zmx00000gp/T/tmpi6hw_nsg/gomoku/checkpoints
Models can be found in /var/folders/0s/fny92xb54c50mb90g3l_zmx00000gp/T/tmpi6hw_nsg/gomoku/models
Logs can be found in /var/folders/0s/fny92xb54c50mb90g3l_zmx00000gp/T/tmpi6hw_nsg/gomoku/logs


---

# Tutorial: Training a Conv Model
#### Detecting Lines of Three on a Board of 7x7

## Sampling Random Boards

In [8]:
from notebooks.ml_basics_recap.data import new_sample
sample = new_sample(board_size=input_size, num_blacks=20, num_whites=0)
sample.shape

(7, 7, 2)

In [9]:
gt.print_channels(sample, True)

shape: (7, 7, 2)
[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 1. 1. 1. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]]


---

## The Detection Map

In [10]:
from notebooks.ml_basics_recap.models.heuristic_detector import HeuristicDetector
detector = HeuristicDetector(input_size)

2022-08-04 14:59:25.216163: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Notice the additional dimension for the batch size come and go

In [11]:
tf.floor(detector(np.expand_dims(sample, 0))+.01)

<tf.Tensor: shape=(1, 7, 7), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)>

In [12]:
np.squeeze(detector(np.expand_dims(sample, 0))).shape

(7, 7)

---

## Batches From a Dataset
#### Strong Advice: Meticulously observe the shape of all incoming data!

In [13]:
BATCH_SIZE=4

In [14]:
from notebooks.ml_basics_recap.data import new_sample, new_dataset
dataset = new_dataset(100, lambda: new_sample(input_size, 20, 0), detector).batch(BATCH_SIZE)

In [15]:
iterator  = iter(dataset)
states, labels = iterator.next()
print (f"Inputs' shape: {states.shape}, Labels' shape: {labels.shape}")

Inputs' shape: (4, 7, 7, 2), Labels' shape: (4, 7, 7)


---

## The Trainable Model

In [16]:
from notebooks.ml_basics_recap.models import SimpleConvQFunction

model_q = SimpleConvQFunction(input_size, n_filters=8, n_layers=4)
print(model_q(states).shape)
print(np.squeeze(model_q(np.expand_dims(states, 0))).shape)

(4, 7, 7, 1)
(4, 7, 7)


---

## Training

In [17]:
TRAIN_SIZE = 1024 * 8
TEST_SIZE = 1024
BATCH_SIZE = 256

### Create a dataset from the heuristics

In [18]:
train_dataset = new_dataset(size=TRAIN_SIZE,
                            sampler=lambda: new_sample(board_size=input_size, num_blacks=20, num_whites=0),
                            labeler=detector, separate=False).batch(BATCH_SIZE)

test_dataset = new_dataset(size=TEST_SIZE,
                           sampler=lambda: new_sample(board_size=input_size, num_blacks=20, num_whites=0),
                           labeler=detector, separate=False).batch(BATCH_SIZE)

In [19]:
import tensorflow as tf

In [20]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

In [21]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.MeanSquaredError('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.MeanSquaredError('test_accuracy')

In [22]:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [23]:
import datetime as dt
current_time = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [24]:
iterator  = iter(train_dataset)
x_train, y_train = iterator.next()

In [25]:
flatten = tf.keras.layers.Flatten()
model = SimpleConvQFunction(input_size, n_filters=8, n_layers=4)
labels = model(x_train, training=True)
loss = loss_object(y_train, labels)
model.summary()

Model: "simple_conv_q_function_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           multiple                  152       
                                                                 
 conv2d_6 (Conv2D)           multiple                  584       
                                                                 
 conv2d_7 (Conv2D)           multiple                  584       
                                                                 
 conv2d_8 (Conv2D)           multiple                  584       
                                                                 
 conv2d_9 (Conv2D)           multiple                  73        
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


In [26]:
EPOCHS = 40

for epoch in range(EPOCHS):
    for x_train, y_train in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for x_test, y_test in test_dataset:
        test_step(model, x_test, y_test)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)


    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.07095175236463547, Accuracy: 7.0951738357543945, Test Loss: 0.06868217885494232, Test Accuracy: 6.868217945098877
Epoch 2, Loss: 0.06382053345441818, Accuracy: 6.382052421569824, Test Loss: 0.06011682376265526, Test Accuracy: 6.011682510375977
Epoch 3, Loss: 0.05244046077132225, Accuracy: 5.244045734405518, Test Loss: 0.047671493142843246, Test Accuracy: 4.767148971557617
Epoch 4, Loss: 0.04434274137020111, Accuracy: 4.434273719787598, Test Loss: 0.04266763851046562, Test Accuracy: 4.2667646408081055
Epoch 5, Loss: 0.03900414705276489, Accuracy: 3.900414228439331, Test Loss: 0.036366693675518036, Test Accuracy: 3.636669397354126
Epoch 6, Loss: 0.03286769613623619, Accuracy: 3.2867696285247803, Test Loss: 0.030366241931915283, Test Accuracy: 3.0366241931915283
Epoch 7, Loss: 0.02716732583940029, Accuracy: 2.7167320251464844, Test Loss: 0.02474280074238777, Test Accuracy: 2.4742798805236816
Epoch 8, Loss: 0.021295733749866486, Accuracy: 2.1295740604400635, Test Loss: 0.0

In [27]:
BATCH_SIZE=1

In [28]:
dataset = new_dataset(10, lambda: new_sample(input_size, 20, 0), detector).batch(BATCH_SIZE)
iterator  = iter(dataset)

### Labels vs Learned

In [29]:
states, labels = iterator.next()
pred = np.squeeze(model(states))

gt.print_bin(states, combine=True)
print(tf.floor(labels+.01))
print(np.round(pred, 2))

Deprecated. Use print_channels instead.
tf.Tensor(
[[[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]], shape=(1, 7, 7), dtype=float32)
[[-0.   -0.   -0.   -0.   -0.   -0.    0.  ]
 [ 0.   -0.   -0.   -0.   -0.   -0.   -0.  ]
 [ 0.    0.   -0.   -0.   -0.   -0.   -0.  ]
 [ 0.   -0.    0.    0.   -0.   -0.   -0.  ]
 [ 0.01  0.99 -0.   -0.   -0.   -0.   -0.  ]
 [ 0.01 -0.   -0.01  0.   -0.   -0.   -0.  ]
 [ 0.    0.   -0.01 -0.   -0.   -0.   -0.  ]]


In [30]:
train_log_dir

'logs/gradient_tape/20220804-145952/train'

In [32]:
%tensorboard --logdir logs/gradient_tape/train_log_dir

Launching TensorBoard...